1회차 대비 추가된 점 : 지하철데이터에 우이신설선 데이터 추가, 지하철 구별 카운트 오류슈정&보완<br>
                  1회차에서 사용한 어린이집 데이터는 서울의 25개 구 중에 4개 구가 빠져있었으며,<br>
                  모든 구의 데이터가 들어있는 어린이집 데이터를 구해와서 추가함

# 아파트데이터 추가전처리 및 파생변수 생성과정 함수화

In [107]:
def data_cleaning(path):
    
    import pandas as pd
    import numpy as np
    import time
    
    start = time.time()
    data = pd.read_csv(path, encoding='cp949')

    # 거래금액열에 값들의 ','를 없애기
    data['거래금액(만원)'] = data['거래금액(만원)'].str.replace(',','')
    data['거래금액(만원)'] = data['거래금액(만원)'].astype(int)
    print("========== COMPLETED REPLACE")

    # data 의 시군구 컬럼에서 구이름과 동이름만 가져오기
    data['city'], data['gu'], data['dong'] = data['시군구'].str.split(' ',2).str
    print("========== COMPLETED SPLIT")
        
    # 열이름 영어이름으로 바꿔주기
    data = data.rename(columns={'단지명':'apt','전용면적(㎡)':'exclusive_area',
                                  '계약년월':'transaction_ym','계약일':'transaction_day','거래금액(만원)':'transaction_real_price',
                                  '층':'floor','건축년도':'completion_year','도로명':'addr_kr','위도':'apt_lat','경도':'apt_lng'})
    print("========== COMPLETED RENAME")
    ##### 법정동코드 데이터 적용 #####
    # 법정동 코드 데이터 불러오기
    dong_code = pd.read_csv('data/ProcessedData/dong_code.csv', encoding='cp949')
    # 법정동 코드를 동이름 기준으로 재구성
    dong_code = dong_code[['법정동코드','시군구명','읍면동명']].sort_values(['시군구명','읍면동명'])
    # 열이름 교체
    dong_code.columns = ['code','gu','dong']
    # 데이터에 법정동코드 적용
    data = pd.merge(data,dong_code, how='left')
    print("========== COMPLETED APPLY DONGCODE")
    
    ##### 지하철 데이터 적용 #####
    # 지하철 데이터 불러오기
    subway = pd.read_csv('data/ProcessedData/subway_cnt(UI).csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,subway, how='left')
    print("========== COMPLETED APPLY SUBWAY")
    
    ##### 공원 데이터 적용  #####
    # 공원 데이터 불러오기
    park = pd.read_csv('data/ProcessedData/park_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,park, how='left')
    # 'park_cnt' 결측치 대체
    data.park_cnt = data.park_cnt.fillna(0)
    print("========== COMPLETED APPLY PARK COUNT DATA")
    
    ##### 스타벅스 데이터 적용 #####
    # 스타벅스 데이터 불러오기
    starbucks = pd.read_csv('data/ProcessedData/starbucks_cnt.csv', encoding='cp949')
    # 데이터 적용
    data = pd.merge(data,starbucks, how='left')
    print("========== COMPLETED APPLY STARBUCKS COUNT DATA")
    
    ##### 학원 데이터 적용 #####
    academy = pd.read_csv('data/ProcessedData/academy_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,academy, how='left')
    print("========== COMPLETED APPLY ACADEMY COUNT DATA")
    
    ##### 어린이집 데이터 적용  #####
    # 어린이집 데이터 불러오기
    daycare = pd.read_csv('data/ProcessedData/daycare_cnt.csv', encoding='cp949')
    # 데이터에 적용
    data = pd.merge(data,daycare, how='left')
    # 'daycare_cnt' 결측치 대체
    #data.daycare_cnt = data.daycare_cnt.fillna(0)
    print("========== COMPLETED APPLY DAY CARE CENTER COUNT DATA")
    
    ##### 유치원 데이터 전처리 #####
    kgarden = pd.read_csv('data/ProcessedData/kindergarden_cnt.csv', encoding='cp949')
    data = pd.merge(data,kgarden, how='left')
    print("========== COMPLETED APPLY KINDERGARDEN COUNT DATA")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    #### 초,중,고,특수학교의 구별 빈도수는 어린이집,유치원,초중고(+특수)끼리는 관계가 분명하게 있지만, 아파트거래금액에는 영향이 거의 없음
    #### 그러므로 구별 초중고(+특수) 각각의 학교위치와 구별 아파트위치의 거리를 계산하여 적용 시켜보기로함
    ## 초등학교 데이터 적용 ##
    # element = pd.read_csv('data/ProcessedData/element_cnt.csv', encoding='cp949')
    # train = pd.merge(train,element, how='left')
    ## 중학교 데이터 적용 ##
    # middle = pd.read_csv('data/ProcessedData/middle_cnt.csv', encoding='cp949')
    # train = pd.merge(train,middle, how='left')
    ## 고등학교 데이터 적용 ##
    # high = pd.read_csv('data/ProcessedData/high_cnt.csv', encoding='cp949')
    # train = pd.merge(train,high, how='left')
    ## 특수학교 데이터 적용 ##
    # special = pd.read_csv('data/ProcessedData/special_cnt.csv', encoding='cp949')
    # train = pd.merge(train,special, how='left')
    # train.corr() # 아파트 거래금액과의 상관관계 계수 : -0.03,0.08,-0.01,0.01
    
    return data

In [108]:
train = data_cleaning('data/ProcessedData/newTrain.csv')

========== COMPLETED REPLACE


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  app.launch_new_instance()


========== COMPLETED SPLIT
========== COMPLETED RENAME
========== COMPLETED APPLY DONGCODE
========== COMPLETED APPLY SUBWAY
========== COMPLETED APPLY PARK COUNT DATA
========== COMPLETED APPLY STARBUCKS COUNT DATA
========== COMPLETED APPLY ACADEMY COUNT DATA
========== COMPLETED APPLY DAY CARE CENTER COUNT DATA
========== COMPLETED APPLY KINDERGARDEN COUNT DATA
---------- Done 9.35 sec ----------


In [109]:
test = data_cleaning('data/ProcessedData/newTest.csv')

========== COMPLETED REPLACE


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  app.launch_new_instance()


========== COMPLETED SPLIT
========== COMPLETED RENAME
========== COMPLETED APPLY DONGCODE
========== COMPLETED APPLY SUBWAY
========== COMPLETED APPLY PARK COUNT DATA
========== COMPLETED APPLY STARBUCKS COUNT DATA
========== COMPLETED APPLY ACADEMY COUNT DATA
========== COMPLETED APPLY DAY CARE CENTER COUNT DATA
========== COMPLETED APPLY KINDERGARDEN COUNT DATA
---------- Done 3.41 sec ----------


In [110]:
duplicate_tr = train.copy()
duplicate_tr = duplicate_tr[['gu','dong','addr_kr','apt','apt_lat','apt_lng']].drop_duplicates()
duplicate_tr

,gu,dong,addr_kr,apt,apt_lat,apt_lng
0,구로구,신도림동,신도림로 87,동아1,37.510960,126.882239
1,강동구,암사동,고덕로 131,롯데캐슬퍼스트,37.556344,127.141943
2,관악구,봉천동,관악로 285,봉천동아,37.487642,126.956106
3,은평구,증산동,증산서길 72,칠산(185-2),37.582730,126.906213
4,강동구,암사동,고덕로 131,롯데캐슬퍼스트,37.555167,127.143839
...,...,...,...,...,...,...
792803,송파구,오금동,성내천로6길 9-7,오금아르미안,37.503810,127.135234
792944,강남구,신사동,논현로153길 42,청담쉐르빌,37.520614,127.024757
793030,중랑구,상봉동,상봉중앙로8가길 5,동일스카이시티 101동,37.602002,127.089611
793087,도봉구,쌍문동,노해로 214-1,원산,37.650799,127.031925


In [111]:
duplicate_te = test.copy()
duplicate_te = duplicate_te[['gu','dong','addr_kr','apt','apt_lat','apt_lng']].drop_duplicates()
duplicate_te

,gu,dong,addr_kr,apt,apt_lat,apt_lng
0,성동구,하왕십리동,왕십리로31길 20,한진그랑빌,37.562984,127.026759
1,노원구,공릉동,화랑로 556,효성아파트,37.621153,127.088285
2,도봉구,창동,도봉로136길 28,북한산아이파크5차,37.656571,127.042729
3,성동구,금호동4가,금호로 15,서울숲푸르지오,37.543857,127.022725
4,강동구,성내동,성내로6다길 15,영풍마드레빌,37.527171,127.123143
...,...,...,...,...,...,...
263948,강동구,성내동,강동대로 143,삼성파크타워,37.526598,127.123206
264007,중랑구,중화동,동일로144길 74,한영(101),37.606521,127.082165
264145,강남구,역삼동,테헤란로14길 13,이스턴오피스텔,37.498549,127.033525
264187,종로구,창신동,낙산성곽동길 35-1,(627-120),37.575757,127.008957


In [5]:
import pandas as pd, numpy as np
subway = pd.read_csv('data/ProcessedData/location/subway_loc(UI).csv', encoding='cp949')

from math import cos, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) #2*R*asin...

duplicate_tr['dist_subway'] = np.nan

for i in range(len(duplicate_tr)):
    for j in range(len(subway)):
        # 아파트 위치와 지하철 위치의 거리를 구한다.
        dist = distance(duplicate_tr['apt_lat'].iloc[i],duplicate_tr['apt_lng'].iloc[i],subway['lat'].iloc[j],subway['lng'].iloc[j])
        if dist < duplicate_tr['dist_subway'].iloc[i]:          # 새로 구한 거리가 기존의 거리보다 짧을 때
            duplicate_tr['dist_subway'].iloc[i] = dist          # 새로 구한 거리로 대체
        elif dist > duplicate_tr['dist_subway'].iloc[i]:        # 새로 구한 거리가 기존의 거리보다 길 때
            continue                                            # 다른 거리를 구한다.
        duplicate_tr['dist_subway'].iloc[i] = dist              # <--기준점 거리(처음 구해진 거리)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [83]:
daycare = pd.read_csv('data/ProcessedData/location/daycare_loc.csv', encoding='cp949')
daycare

,gu,name,lat,lng
0,종로구,이화어린이집,37.574945,127.005125
1,종로구,삼청어린이집,37.584006,126.980487
2,종로구,창신어린이집,37.573605,127.012017
3,종로구,아동회관어린이집,37.578307,127.011810
4,종로구,부암어린이집,37.601522,126.963073
...,...,...,...,...
9028,강동구,구립롯데캐슬베네루체어린이집,37.566470,126.977963
9029,강동구,아이뜰어린이집,37.552539,127.167909
9030,강동구,구립아르테온보나어린이집,37.566470,126.977963
9031,강동구,구립아르테온보예어린이집,37.555435,127.168334


In [89]:
import pandas as pd, numpy as np
daycare = pd.read_csv('data/ProcessedData/location/daycare_loc.csv', encoding='cp949')

from math import cos, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) #2*R*asin...

duplicate_tr['dist_daycare'] = np.nan

for i in range(len(duplicate_tr)):
    for j in range(len(daycare)):
        # 아파트 위치와 지하철 위치의 거리를 구한다.
        dist = distance(duplicate_tr['apt_lat'].iloc[i],duplicate_tr['apt_lng'].iloc[i],daycare['lat'].iloc[j],daycare['lng'].iloc[j])
        if dist < duplicate_tr['dist_daycare'].iloc[i]:          # 새로 구한 거리가 기존의 거리보다 짧을 때
            duplicate_tr['dist_daycare'].iloc[i] = dist          # 새로 구한 거리로 대체
        elif dist > duplicate_tr['dist_daycare'].iloc[i]:        # 새로 구한 거리가 기존의 거리보다 길 때
            continue                                             # 다른 거리를 구한다.
        duplicate_tr['dist_daycare'].iloc[i] = dist              # <--기준점 거리(처음 구해진 거리)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [90]:
duplicate_tr

,gu,dong,addr_kr,apt,apt_lat,apt_lng,dist_daycare
0,강남구,개포동,언주로 103,개포2차현대아파트(220),37.481344,127.052912,8.267502
4,강남구,개포동,언주로 3,개포6차우성아파트1동~8동,37.476792,127.055926,8.389665
11,강남구,개포동,개포로 307,개포우성3차,37.484631,127.056092,7.821907
14,강남구,개포동,개포로109길 69,개포자이(12-2),37.496292,127.076127,5.630132
18,강남구,개포동,삼성로 11,개포주공 3단지,37.484135,127.068092,7.046355
...,...,...,...,...,...,...,...
799514,중구,황학동,난계로13길 36,로얄,37.567290,127.021571,3.035362
800332,중랑구,면목동,사가정로42길 90,두산5,37.577959,127.081093,4.730603
800651,중랑구,면목동,동일로101길 63,한신,37.588544,127.076312,5.797044
801479,중랑구,상봉동,상봉로 131,상봉 듀오트리스,37.597407,127.092178,5.676174


In [93]:
train = pd.merge(train,duplicate_tr,how='left')
train.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,gu,dong,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_daycare
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,강남구,개포동,1168010300,27,134.0,84,6789,338,40,8.267502


### 특정 장소와 거리구하기 함수화

In [105]:
def calculate_distance(data):
    
    import time
    
    func_start = time.time()
    ##### 자료 준비 #####
    import pandas as pd
    river = pd.read_csv('data/ProcessedData/location/river_loc.csv', encoding='cp949')
    subway = pd.read_csv('data/ProcessedData/location/subway_loc(UI).csv', encoding='cp949')
    park = pd.read_csv('data/ProcessedData/location/park_loc.csv', encoding='cp949')
    starbucks = pd.read_csv('data/ProcessedData/location/starbucks_loc.csv', encoding='cp949')
    '''kgarden = pd.read_csv('data/ProcessedData/location/kindergarden_loc.csv', encoding='cp949')
    element = pd.read_csv('data/ProcessedData/location/element_loc.csv', encoding='cp949')
    middle = pd.read_csv('data/ProcessedData/location/middle_loc.csv', encoding='cp949')
    high = pd.read_csv('data/ProcessedData/location/high_loc.csv', encoding='cp949')'''
    print("data is ready to calculate")
    
    ##### 거리계산 함수 #####
    from math import cos, asin, sqrt
    def distance(lat1, lon1, lat2, lon2):
        p = 0.017453292519943295     #Pi/180
        a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
        return 12742 * asin(sqrt(a)) #2*R*asin...
    print("func of distance is working.....")
    
    ##### 거리열 미리 추가 #####
    import numpy as np
    data['dist_river'] = np.nan
    data['dist_subway'] = np.nan
    data['dist_park'] = np.nan
    data['dist_starbucks'] = np.nan
    '''data['dist_kgarden'] = np.nan
    data['dist_element'] = np.nan
    data['dist_middle'] = np.nan
    data['dist_high'] = np.nan'''
    print("each of distance columns are created")
    
    start = time.time()
    # < 1 > 한강과의 최단 거리
    for i in range(len(data)):
        for j in range(len(river)):
            # 아파트 위치와 한강의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],river['lat'].iloc[j],river['lng'].iloc[j])
            if dist < data['dist_river'].iloc[i]:           # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_river'].iloc[i] = dist           # 새로 구한 거리로 대체
            elif dist > data['dist_river'].iloc[i]:         # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_river'].iloc[i] = dist               # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF 'HAN'RIVER BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 2 > 지하철과의 최단 거리
    for i in range(len(data)):
        for j in range(len(subway)):
            # 아파트 위치와 지하철 위치의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],subway['lat'].iloc[j],subway['lng'].iloc[j])
            if dist < data['dist_subway'].iloc[i]:          # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_subway'].iloc[i] = dist          # 새로 구한 거리로 대체
            elif dist > data['dist_subway'].iloc[i]:        # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_subway'].iloc[i] = dist              # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF SUBWAY BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    # < 3 > 공원과의 최단 거리
    start = time.time()
    for i in range(len(data)):
        for j in range(len(park)):
            # 아파트 위치와 공원의 거리를 구한다.
            dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],park['lat'].iloc[j],park['lng'].iloc[j])
            if dist < data['dist_park'].iloc[i]:            # 새로 구한 거리가 기존의 거리보다 짧을 때
                data['dist_park'].iloc[i] = dist            # 새로 구한 거리로 대체
            elif dist > data['dist_park'].iloc[i]:          # 새로 구한 거리가 기존의 거리보다 길 때
                continue                                    # 다른 거리를 구한다.
            data['dist_park'].iloc[i] = dist           # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF PARK BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 4 > 스타벅스와의 최단 거리
    for i in range(len(data)):
        for j in range(len(starbucks)):
            if data['gu'].iloc[i] == starbucks['gu'].iloc[j]:
                # 아파트 위치와 스타벅스 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],starbucks['lat'].iloc[j],starbucks['lng'].iloc[j])
                if dist < data['dist_starbucks'].iloc[i]:       # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_starbucks'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_starbucks'].iloc[i]:     # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                                    # 다른 거리를 구한다.
                data['dist_starbucks'].iloc[i] = dist           # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF STARBUCKS BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    print('---------- Done All Function {:.2f} sec ----------'.format(time.time() - func_start))
    
    '''start = time.time()
    # < 5 > 유치원과의 최단 거리
    for i in range(len(data)):
        for j in range(len(kgarden)):
            if data['gu'].iloc[i] == kgarden['gu'].iloc[j]:
                # 아파트 위치와 유치원 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],kgarden['lat'].iloc[j],kgarden['lng'].iloc[j])
                if dist < data['dist_kgarden'].iloc[i]:    # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_kgarden'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_kgarden'].iloc[i]:  # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                             # 다른 거리를 구한다.
                data['dist_kgarden'].iloc[i] = dist # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF KINDERGARDEN BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 6 > 초등학교와의 최단 거리
    for i in range(len(data)):
        for j in range(len(element)):
            if data['gu'].iloc[i] == element['gu'].iloc[j]:
                # 아파트 위치와 초등학교 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],element['lat'].iloc[j],element['lng'].iloc[j])
                if dist < data['dist_element'].iloc[i]:    # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_element'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_element'].iloc[i]:  # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                             # 다른 거리를 구한다.
                data['dist_element'].iloc[i] = dist # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF ELEMENTARY SCHOOL BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 7 > 중학교와의 최단 거리
    for i in range(len(data)):
        for j in range(len(middle)):
            if data['gu'].iloc[i] == middle['gu'].iloc[j]:
                # 아파트 위치와 중학교 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],middle['lat'].iloc[j],middle['lng'].iloc[j])
                if dist < data['dist_middle'].iloc[i]:    # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_middle'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_middle'].iloc[i]:  # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                             # 다른 거리를 구한다.
                data['dist_middle'].iloc[i] = dist # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF MIDDLE SCHOOL BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))
    
    start = time.time()
    # < 8 > 고등학교와의 최단 거리
    for i in range(len(data)):
        for j in range(len(high)):
            if data['gu'].iloc[i] == high['gu'].iloc[j]:
                # 아파트 위치와 고등학교 위치의 거리를 구한다.
                dist = distance(data['apt_lat'].iloc[i],data['apt_lng'].iloc[i],high['lat'].iloc[j],high['lng'].iloc[j])
                if dist < data['dist_high'].iloc[i]:    # 새로 구한 거리가 기존의 거리보다 짧을 때
                    data['dist_high'].iloc[i] = dist       # 새로 구한 거리로 대체
                elif dist > data['dist_high'].iloc[i]:  # 새로 구한 거리가 기존의 거리보다 길 때
                    continue                             # 다른 거리를 구한다.
                data['dist_high'].iloc[i] = dist # <--기준점 거리(처음 구해진 거리)
    print("========== COMPLETED DINTANCE OF HIGH SCHOOL BETWEEN APT")
    print('---------- Done {:.2f} sec ----------'.format(time.time() - start))'''
    
    return data

In [159]:
duplicate_tr = calculate_distance(duplicate_tr)

In [ ]:
duplicate_te = calculate_distance(duplicate_te)

In [518]:
train = pd.merge(train,duplicate_tr,how='left')
train.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,28,134.0,84,6789,223.0,40,4.510525,0.805268,0.45746,0.792201


In [521]:
test = pd.merge(test,duplicate_te,how='left')
test.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,201707,8,89900,1,1988,...,28,134.0,84,6789,223.0,40,4.410927,0.613785,0.330724,0.752602


# 각종 세권 평균매매가격 구하기 함수화

In [541]:
def influence_area(data):
    import numpy as np
    # 역세권
    super_SIA = int(np.round(data.query("dist_subway <= 0.2").transaction_real_price.values.mean(),0))
    SIA_1st = int(np.round(data.query("dist_subway > 0.2 & dist_subway <= 0.5").transaction_real_price.values.mean(),0))
    SIA_2nd = int(np.round(data.query("dist_subway > 0.5 & dist_subway <= 1.0").transaction_real_price.values.mean(),0))
    not_SIA = int(np.round(data.query("dist_subway > 1").transaction_real_price.values.mean(),0))
    
    data.loc[data.dist_subway <= 0.2, "StationIA(mean_price)"] = super_SIA
    data.loc[(data.dist_subway > 0.2)&(data.dist_subway <= 0.5), "StationIA(mean_price)"] = SIA_1st
    data.loc[(data.dist_subway > 0.5)&(data.dist_subway <= 1), "StationIA(mean_price)"] = SIA_2nd
    data.loc[data.dist_subway > 1, "StationIA(mean_price)"] = not_SIA

    # 스(타벅스)세권
    starbucks_1st = int(np.round(data.query("dist_starbucks <= 0.2").transaction_real_price.values.mean(),0))
    starbucks_2nd = int(np.round(data.query("dist_starbucks > 0.2 & dist_starbucks <= 0.5").transaction_real_price.values.mean(),0))
    starbucks_3rd = int(np.round(data.query("dist_starbucks > 0.5 & dist_starbucks <= 1.0").transaction_real_price.values.mean(),0))
    starbucks_4th = int(np.round(data.query("dist_starbucks > 1").transaction_real_price.values.mean(),0))
    
    data.loc[data.dist_starbucks <= 0.2, "StarbucksIA(mean_price)"] = starbucks_1st
    data.loc[(data.dist_starbucks > 0.2)&(data.dist_starbucks <= 0.5), "StarbucksIA(mean_price)"] = starbucks_2nd
    data.loc[(data.dist_starbucks > 0.5)&(data.dist_starbucks <= 1), "StarbucksIA(mean_price)"] = starbucks_3rd
    data.loc[data.dist_starbucks > 1, "StarbucksIA(mean_price)"] = starbucks_4th

    # 숲세권
    park_1st = int(np.round(data.query("dist_park <= 0.2").transaction_real_price.values.mean(),0))
    park_2nd = int(np.round(data.query("dist_park > 0.2 & dist_park <= 0.5").transaction_real_price.values.mean(),0))
    park_3rd = int(np.round(data.query("dist_park > 0.5 & dist_park <= 1.0").transaction_real_price.values.mean(),0))
    park_4th = int(np.round(data.query("dist_park > 1").transaction_real_price.values.mean(),0))

    data.loc[data.dist_park <= 0.2, "ParkIA(mean_price)"] = park_1st
    data.loc[(data.dist_park > 0.2)&(data.dist_park <= 0.5), "ParkIA(mean_price)"] = park_2nd
    data.loc[(data.dist_park > 0.5)&(data.dist_park <= 1), "ParkIA(mean_price)"] = park_3rd
    data.loc[data.dist_park > 1, "ParkIA(mean_price)"] = park_4th
    
    return data

In [542]:
train = influence_area(train)
train.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price)
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,200606,1,60000,4,1988,...,6789,223.0,40,4.510525,0.805268,0.45746,0.792201,44582.0,41268.0,46995.0


In [543]:
test = influence_area(test)
test.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price)
0,서울특별시 강남구 개포동,655,2,개포2차현대아파트(220),77.75,201707,8,89900,1,1988,...,6789,223.0,40,4.410927,0.613785,0.330724,0.752602,68673.0,64696.0,73303.0


In [219]:
apt = pd.read_csv('visualization/processedData.csv', encoding='cp949')
apt.head(1)

,시군구,본번,부번,apt,exclusive_area,transaction_ym,transaction_day,transaction_real_price,floor,completion_year,...,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price)
0,서울특별시 구로구 신도림동,643,0,동아1,84.908,201611,8,50000,1,1999,...,1281,473,38,3.191675,0.372833,0.548902,0.629231,52499.0,46844.0,44941.0


In [218]:
apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057565 entries, 0 to 1057564
Data columns (total 32 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   시군구                      1057565 non-null  object 
 1   본번                       1057565 non-null  int64  
 2   부번                       1057565 non-null  int64  
 3   apt                      1057565 non-null  object 
 4   exclusive_area           1057565 non-null  float64
 5   transaction_ym           1057565 non-null  int64  
 6   transaction_day          1057565 non-null  int64  
 7   transaction_real_price   1057565 non-null  int64  
 8   floor                    1057565 non-null  int64  
 9   completion_year          1057565 non-null  int64  
 10  addr_kr                  1057565 non-null  object 
 11  apt_lat                  1057565 non-null  float64
 12  apt_lng                  1057565 non-null  float64
 13  city                     1057565 non-null 

In [220]:
apt['transaction_ymd'] = (apt.transaction_ym * 100) + apt.transaction_day

In [221]:
# 드랍할 컬럼 설정
obj = ['시군구','본번','부번','apt','transaction_ym','transaction_day','transaction_y','transaction_m','completion_year','addr_kr','city','gu','dong']
apt = apt.drop(obj, axis=1)
apt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057565 entries, 0 to 1057564
Data columns (total 20 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   exclusive_area           1057565 non-null  float64
 1   transaction_real_price   1057565 non-null  int64  
 2   floor                    1057565 non-null  int64  
 3   apt_lat                  1057565 non-null  float64
 4   apt_lng                  1057565 non-null  float64
 5   code                     1057565 non-null  int64  
 6   subway_cnt               1057565 non-null  int64  
 7   park_cnt                 1057565 non-null  float64
 8   starbucks_cnt            1057565 non-null  int64  
 9   academy_cnt              1057565 non-null  int64  
 10  daycare_cnt              1057565 non-null  int64  
 11  kindergarden_cnt         1057565 non-null  int64  
 12  dist_river               1057565 non-null  float64
 13  dist_subway              1057565 non-null 

In [232]:
corr = apt.corr()
corr_trsc = corr.transaction_real_price.sort_values(ascending=False).to_frame()
corr_trsc.style.background_gradient(cmap='magma')

,transaction_real_price
transaction_real_price,1.000000
exclusive_area,0.630778
starbucks_cnt,0.478429
academy_cnt,0.406577
subway_cnt,0.374043
park_cnt,0.335678
transaction_ymd,0.290544
code,0.260978
StarbucksIA(mean_price),0.258125
floor,0.151195


### 학습 데이터, 예측 데이터 나누기

In [222]:
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(42)

label = apt.transaction_real_price
feature = apt.drop('transaction_real_price', axis=1)

train, pred, target, answer = train_test_split(feature, label, test_size=0.25, random_state=42)
train.shape, target.shape, pred.shape, answer.shape

((793173, 19), (793173,), (264392, 19), (264392,))

### 모델 알고리즘 설정

In [182]:
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

lgb = LGBMRegressor(random_state=0, max_depth=-1, min_child_samples=0,min_data=0,
                               min_child_weight=1, min_data_in_bin=1, min_split_gain=0.0,
                               n_estimators=600, n_jobs=-1, num_class=1,num_leaves=10)

xgb = XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,learning_rate=0.05, max_depth=3, 
                         min_child_weight=1.7817, n_estimators=2200,reg_alpha=0.4640, reg_lambda=0.8571,
                         subsample=0.5213, silent=1,random_state =7, nthread = -1,n_jobs=-1)

gb = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,max_depth=4, max_features='sqrt',
                              min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =5)

cat = CatBoostRegressor(iterations=5000, learning_rate=0.2, depth=5, loss_function='RMSE', verbose=True)

### 데이터 학습

In [ ]:
model_lgb = lgb.fit(train, target)
model_xgb = xgb.fit(train, target)
model_gb = gb.fit(train, target)
model_cat = cat.fit(train, target)

In [223]:
model_lgb = lgb.fit(train, target)

In [278]:
model_xgb = xgb.fit(train, target)

In [297]:
model_gb = gb.fit(train, target)

In [234]:
model_cat = cat.fit(train, target)

0:	learn: 33899.2106392	total: 134ms	remaining: 11m 8s
1:	learn: 30382.7633236	total: 199ms	remaining: 8m 16s
2:	learn: 27615.4106642	total: 266ms	remaining: 7m 23s
3:	learn: 25551.3866363	total: 329ms	remaining: 6m 51s
4:	learn: 23945.5252610	total: 387ms	remaining: 6m 26s
5:	learn: 22613.9887276	total: 440ms	remaining: 6m 6s
6:	learn: 21505.6505866	total: 489ms	remaining: 5m 49s
7:	learn: 20575.6323907	total: 534ms	remaining: 5m 33s
8:	learn: 19886.5273289	total: 581ms	remaining: 5m 22s
9:	learn: 19300.2712299	total: 634ms	remaining: 5m 16s
10:	learn: 18874.2964040	total: 689ms	remaining: 5m 12s
11:	learn: 18433.2158499	total: 732ms	remaining: 5m 4s
12:	learn: 18166.0834539	total: 773ms	remaining: 4m 56s
13:	learn: 17870.2822902	total: 818ms	remaining: 4m 51s
14:	learn: 17546.1938417	total: 871ms	remaining: 4m 49s
15:	learn: 17282.0452232	total: 929ms	remaining: 4m 49s
16:	learn: 17091.5248136	total: 989ms	remaining: 4m 49s
17:	learn: 16885.6894520	total: 1.04s	remaining: 4m 47s
18:	

### 전개

In [298]:
pred.head()

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_ymd
152727,101.860,11,37.615754,127.093753,1126010600,11,54.0,6,819,348,32,6.678683,0.268030,0.225396,1.163364,52499.0,38805.0,53214.0,20170511
839530,114.260,4,37.586831,126.880713,1144012700,15,81.0,30,1932,356,28,1.950846,1.471988,0.081114,1.104869,37834.0,38805.0,52105.0,20110119
137278,168.650,6,37.502390,126.995965,1165010700,15,114.0,48,3390,315,25,0.817041,0.114000,0.315635,0.416012,57909.0,59809.0,53214.0,20130523
259253,59.828,5,37.505240,126.933134,1159010200,13,52.0,8,1374,314,34,1.079989,0.532043,0.482533,0.602319,50160.0,46844.0,53214.0,20100108
706358,53.460,11,37.512095,126.860291,1147010100,6,96.0,11,3186,462,50,4.497167,0.488736,0.099247,1.204707,52499.0,38805.0,52105.0,20180818


In [296]:
pred = pred.drop("transaction_real_price", axis=1)
pred.head()

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_ymd
152727,101.860,11,37.615754,127.093753,1126010600,11,54.0,6,819,348,32,6.678683,0.268030,0.225396,1.163364,52499.0,38805.0,53214.0,20170511
839530,114.260,4,37.586831,126.880713,1144012700,15,81.0,30,1932,356,28,1.950846,1.471988,0.081114,1.104869,37834.0,38805.0,52105.0,20110119
137278,168.650,6,37.502390,126.995965,1165010700,15,114.0,48,3390,315,25,0.817041,0.114000,0.315635,0.416012,57909.0,59809.0,53214.0,20130523
259253,59.828,5,37.505240,126.933134,1159010200,13,52.0,8,1374,314,34,1.079989,0.532043,0.482533,0.602319,50160.0,46844.0,53214.0,20100108
706358,53.460,11,37.512095,126.860291,1147010100,6,96.0,11,3186,462,50,4.497167,0.488736,0.099247,1.204707,52499.0,38805.0,52105.0,20180818


## LightGBM Regressor

In [291]:
## LightGBM Regressor
pred['transaction_real_price'] = np.round(model_lgb.predict(pred),0)  # 예측
error_val = answer - pred.transaction_real_price          # 오차값
print("MAE :", abs(error_val).mean())                     # MAE
print("RMSE :", np.sqrt((error_val ** 2).mean()))         # RMSE
error_rat = error_val / answer * 100                      # 오차율
print("오차율 MAE :", abs(error_rat).mean())               # 오차율 MAE
print("오차율 RMSE :", np.sqrt((error_rat ** 2).mean()))   # 오차율 RMSE
print(abs(error_rat).describe())                          # 오차율 요약 통계량

평균 오차값 : 6096.499610426942
평균 오차율 : 13.386134176393407
count    264392.000000
mean         13.386134
std          26.526076
min           0.000000
25%           4.491121
50%           9.617872
75%          17.056535
max        4861.090909
Name: transaction_real_price, dtype: float64


In [293]:
len(error_rat[error_rat == 0])  # 오차율 0으로 정확하게 예측한 데이터의 개수

18

In [294]:
len(error_rat[error_rat <= 9.617872])  # 오차율 최소값부터 2분위수까지 개수

205380

### 오차율 최대값 데이터

In [240]:
error_rat[error_rat <= -4861]  # 오차율 최대값

856547   -4861.090909
Name: transaction_real_price, dtype: float64

In [241]:
pred.query("index == 856547")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_ymd,transaction_real_price
856547,114.76,10,37.660342,127.065782,1135010500,13,2.0,9,2241,659,70,12.213318,0.688312,1.34746,0.686404,50160.0,46844.0,33882.0,20090811,54572.0


In [245]:
accurate = pd.read_csv('visualization/processedData.csv', encoding='cp949')
accurate.query("index == 856547").T

,856547
시군구,서울특별시 노원구 상계동
본번,1283
부번,0
apt,성원상떼빌
exclusive_area,114.76
transaction_ym,200908
transaction_day,11
transaction_real_price,1100
floor,10
completion_year,1999


In [246]:
accurate.query("addr_kr == '노원로34길 34'")[['시군구','apt','addr_kr','transaction_ym','transaction_day','transaction_real_price','apt_lat','apt_lng']].sort_values('transaction_real_price')

,시군구,apt,addr_kr,transaction_ym,transaction_day,transaction_real_price,apt_lat,apt_lng
856547,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200908,11,1100,37.660342,127.065782
547976,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200608,10,13500,37.660342,127.065782
433782,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200609,28,13900,37.660342,127.065782
567143,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200610,20,14000,37.660342,127.065782
461909,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200610,14,14000,37.660342,127.065782
...,...,...,...,...,...,...,...,...
645213,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201805,28,45000,37.660331,127.065764
469076,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,202001,8,45500,37.660342,127.065782
385025,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201804,21,45700,37.660331,127.065764
931712,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201810,27,49800,37.660331,127.065764


## XGB Regressor

In [280]:
## XGB Regressor
pred['transaction_real_price'] = np.round(model_xgb.predict(pred),0)  # 예측
error_val = answer - pred.transaction_real_price          # 오차값
print("MAE :", abs(error_val).mean())                     # MAE
print("RMSE :", np.sqrt((error_val ** 2).mean()))         # RMSE
error_rat = error_val / answer * 100                      # 오차율
print("오차율 MAE :", abs(error_rat).mean())               # 오차율 MAE
print("오차율 RMSE :", np.sqrt((error_rat ** 2).mean()))   # 오차율 RMSE
print(abs(error_rat).describe())                          # 오차율 요약 통계량

평균 오차값 : 6654.59130859375
평균 오차율 : 14.53962516784668
count    264392.000000
mean         14.539625
std          27.404728
min           0.000000
25%           4.845459
50%          10.359811
75%          18.537548
max        4669.390137
Name: transaction_real_price, dtype: float64


In [282]:
len(error_rat[error_rat == 0])  # 오차율 0으로 정확하게 예측한 데이터의 개수

23

In [283]:
len(error_rat[error_rat <= 10.359811])  # 오차율 최소값부터 2분위수까지 개수

204701

### 오차율 최대값 데이터

In [285]:
error_rat[error_rat <= -4669]  # 오차율 최대값

1040260   -4669.390137
Name: transaction_real_price, dtype: float32

In [286]:
pred.query("index == 1040260")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_ymd,transaction_real_price
1040260,100.53,6,37.612928,126.913113,1138010500,12,98.0,10,1563,504,50,5.949202,0.404792,0.299448,0.422093,52499.0,59809.0,53214.0,20081201,39109.0


In [287]:
accurate = pd.read_csv('visualization/processedData.csv', encoding='cp949')
accurate.query("index == 1040260").T

,1040260
시군구,서울특별시 은평구 구산동
본번,199
부번,24
apt,신호(199-24)
exclusive_area,100.53
transaction_ym,200812
transaction_day,1
transaction_real_price,820
floor,6
completion_year,2008


In [288]:
accurate.query("addr_kr == '서오릉로 181'")[['시군구','apt','addr_kr','transaction_ym','transaction_day','transaction_real_price','apt_lat','apt_lng']].sort_values('transaction_real_price')

,시군구,apt,addr_kr,transaction_ym,transaction_day,transaction_real_price,apt_lat,apt_lng
929040,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,630,37.612928,126.913113
585397,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,660,37.612928,126.913113
276894,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,710,37.612928,126.913113
618127,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,710,37.612928,126.913113
388794,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,730,37.612928,126.913113
364101,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,730,37.612928,126.913113
244770,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,790,37.612928,126.913113
228563,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,800,37.612928,126.913113
109101,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,810,37.612928,126.913113
1040260,서울특별시 은평구 구산동,신호(199-24),서오릉로 181,200812,1,820,37.612928,126.913113


## GradientBoosting Regressor

In [299]:
## GradientBoosting Regressor
pred['transaction_real_price'] = np.round(model_gb.predict(pred),0)   # 예측
error_val = answer - pred.transaction_real_price          # 오차값
print("MAE :", abs(error_val).mean())                     # MAE
print("RMSE :", np.sqrt((error_val ** 2).mean()))         # RMSE
error_rat = error_val / answer * 100                      # 오차율
print("오차율 MAE :", abs(error_rat).mean())               # 오차율 MAE
print("오차율 RMSE :", np.sqrt((error_rat ** 2).mean()))   # 오차율 RMSE
print(abs(error_rat).describe())                          # 오차율 요약 통계량

평균 오차값 : 4879.635677327605
평균 오차율 : 10.20980202439856
count    264392.000000
mean         10.209802
std          24.405843
min           0.000000
25%           3.121904
50%           6.804679
75%          12.612865
max        4941.090909
Name: transaction_real_price, dtype: float64


In [301]:
len(error_rat[error_rat == 0])  # 오차율 0으로 정확하게 예측한 데이터의 개수

33

In [302]:
len(error_rat[error_rat <= 6.804679])  # 오차율 최소값부터 1분위수까지 개수

201491

### 오차율 최대값 데이터

In [303]:
error_rat[error_rat <= -4941]  # 오차율 최대값

856547   -4941.090909
Name: transaction_real_price, dtype: float64

In [304]:
pred.query("index == 856547")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_ymd,transaction_real_price
856547,114.76,10,37.660342,127.065782,1135010500,13,2.0,9,2241,659,70,12.213318,0.688312,1.34746,0.686404,50160.0,46844.0,33882.0,20090811,55452.0


In [305]:
accurate = pd.read_csv('visualization/processedData.csv', encoding='cp949')
accurate.query("index == 856547").T

,856547
시군구,서울특별시 노원구 상계동
본번,1283
부번,0
apt,성원상떼빌
exclusive_area,114.76
transaction_ym,200908
transaction_day,11
transaction_real_price,1100
floor,10
completion_year,1999


In [306]:
accurate.query("addr_kr == '노원로34길 34'")[['시군구','apt','addr_kr','transaction_ym','transaction_day','transaction_real_price','apt_lat','apt_lng']].sort_values('transaction_real_price')

,시군구,apt,addr_kr,transaction_ym,transaction_day,transaction_real_price,apt_lat,apt_lng
856547,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200908,11,1100,37.660342,127.065782
547976,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200608,10,13500,37.660342,127.065782
433782,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200609,28,13900,37.660342,127.065782
567143,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200610,20,14000,37.660342,127.065782
461909,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200610,14,14000,37.660342,127.065782
...,...,...,...,...,...,...,...,...
645213,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201805,28,45000,37.660331,127.065764
469076,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,202001,8,45500,37.660342,127.065782
385025,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201804,21,45700,37.660331,127.065764
931712,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201810,27,49800,37.660331,127.065764


## CatBoost Regressor

In [251]:
## CatBoost Regressor
pred['transaction_real_price'] = np.round(model_cat.predict(pred),0)  # 예측
error_val = answer - pred.transaction_real_price          # 오차값
print("MAE :", abs(error_val).mean())                     # MAE
print("RMSE :", np.sqrt((error_val ** 2).mean()))         # RMSE
error_rat = error_val / answer * 100                      # 오차율
print("오차율 MAE :", abs(error_rat).mean())               # 오차율 MAE
print("오차율 RMSE :", np.sqrt((error_rat ** 2).mean()))   # 오차율 RMSE
print(abs(error_rat).describe())                          # 오차율 요약 통계량

평균 오차값 : 3192.399399376683
평균 오차율 : 7.065177832609855
count    264392.000000
mean          7.065178
std          19.517176
min           0.000000
25%           2.147951
50%           4.679664
75%           8.694899
max        4479.272727
Name: transaction_real_price, dtype: float64


In [268]:
len(error_rat[error_rat == 0])  # 오차율 0으로 정확하게 예측한 데이터의 인덱스

43

In [254]:
len(error_rat[error_rat <= 8.694899])  # 오차율 최소값부터 3분위수까지 개수

234687

### 오차율 최대값 데이터

In [255]:
error_rat[error_rat <= -4479]  # 오차율 최대값

856547   -4479.272727
Name: transaction_real_price, dtype: float64

In [256]:
pred.query("index == 856547")

,exclusive_area,floor,apt_lat,apt_lng,code,subway_cnt,park_cnt,starbucks_cnt,academy_cnt,daycare_cnt,kindergarden_cnt,dist_river,dist_subway,dist_park,dist_starbucks,StationIA(mean_price),StarbucksIA(mean_price),ParkIA(mean_price),transaction_ymd,transaction_real_price
856547,114.76,10,37.660342,127.065782,1135010500,13,2.0,9,2241,659,70,12.213318,0.688312,1.34746,0.686404,50160.0,46844.0,33882.0,20090811,50372.0


In [257]:
accurate = pd.read_csv('visualization/processedData.csv', encoding='cp949')
accurate.query("index == 856547").T

,856547
시군구,서울특별시 노원구 상계동
본번,1283
부번,0
apt,성원상떼빌
exclusive_area,114.76
transaction_ym,200908
transaction_day,11
transaction_real_price,1100
floor,10
completion_year,1999


In [258]:
accurate.query("addr_kr == '노원로34길 34'")[['시군구','apt','addr_kr','transaction_ym','transaction_day','transaction_real_price','apt_lat','apt_lng']].sort_values('transaction_real_price')

,시군구,apt,addr_kr,transaction_ym,transaction_day,transaction_real_price,apt_lat,apt_lng
856547,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200908,11,1100,37.660342,127.065782
547976,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200608,10,13500,37.660342,127.065782
433782,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200609,28,13900,37.660342,127.065782
567143,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200610,20,14000,37.660342,127.065782
461909,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,200610,14,14000,37.660342,127.065782
...,...,...,...,...,...,...,...,...
645213,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201805,28,45000,37.660331,127.065764
469076,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,202001,8,45500,37.660342,127.065782
385025,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201804,21,45700,37.660331,127.065764
931712,서울특별시 노원구 상계동,성원상떼빌,노원로34길 34,201810,27,49800,37.660331,127.065764


In [307]:
pred.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264392 entries, 152727 to 577563
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   exclusive_area           264392 non-null  float64
 1   floor                    264392 non-null  int64  
 2   apt_lat                  264392 non-null  float64
 3   apt_lng                  264392 non-null  float64
 4   code                     264392 non-null  int64  
 5   subway_cnt               264392 non-null  int64  
 6   park_cnt                 264392 non-null  float64
 7   starbucks_cnt            264392 non-null  int64  
 8   academy_cnt              264392 non-null  int64  
 9   daycare_cnt              264392 non-null  int64  
 10  kindergarden_cnt         264392 non-null  int64  
 11  dist_river               264392 non-null  float64
 12  dist_subway              264392 non-null  float64
 13  dist_park                264392 non-null  float64
 14 

### 종합 평가

In [312]:
(4861.090909+4669.390137+4941.090909+4479.272727)/4

4737.7111705

In [311]:
234687/254674*100

92.15192756229533

In [728]:
<평가차수 : 2회차 / 평가일 : 20.06.10 / 학습&예측 데이터 열 개수 19개>

<파생변수 : 아파트 위도&경도, 법정동코드, 구별 지하철&공원&학원&어린이집&유치원&스타벅스 빈도수,
    한강&지하철&공원&스타벅스까지의 최소거리(각 아파트마다), 스(타벅스)세권&역세권&숲세권, 거래날짜(연,월,일) / 17개>

<사용 알고리즘>
1)         LightGBM Regressor : 평균 오차율 13.39% / [유효 예측값 : 2분위수  9.62% / 개수 205380개, 80.64%]
2)              XGB Regressor : 평균 오차율 14.54% / [유효 예측값 : 2분위수 10.36% / 개수 204701개, 80.38%]
3) GradientBoosting Regressor : 평균 오차율 10.21% / [유효 예측값 : 2분위수  6.80% / 개수 201491개, 79.12%]
4)         CatBoost Regressor : 평균 오차율  7.07% / [유효 예측값 : 3분위수  8.69% / 개수 234687개, 92.15%] **최우수 모델

*부가 설명 : 각 알고리즘마다 평균적으로 약 (-)4737% 정도의 최대 오차율을 가진 값들이 존재하는데, 이를 분석해본 결과
    1.오차율 최대값을 가지는 아파트의 실제 거래금액이 동일한 아파트이름을 가진 다른 데이터들 보다 심각하게 낮음 ex)1100(만원), 14000(만원)
    2.추측컨데 사건사고가 있던 세대이거나, 기타 복지정책에 의해 책정된 가격으로 생각됨

실제값 - 예측값 = 오차값

오차값(절대값).mean() = 오차값들의 평균

( 실제값 - 예측값 ) / 실제값 * 100 = 오차율[ (+) : 실제값 > 예측값, (-) : 실제값 < 예측값 ]

오차율(절대값).mean() = 오차율들의 평균

In [14]:
import numpy as np
y = np.array([40,50,48,55])
x = np.array([41.8,46.1,50.4,54.7])

In [15]:
y-x # 각 오차값

array([-1.8,  3.9, -2.4,  0.3])

In [26]:
(y-x)/y*100 # 각 오차율

array([-4.5       ,  7.8       , -5.        ,  0.54545455])

In [25]:
abs((y-x)/y*100).mean() # 평균 오차율

4.4613636363636315

In [37]:
np.sqrt(((y-x)**2).sum() * 0.25) # RSME

2.464751508773246

In [38]:
from sklearn.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(y, x))

2.464751508773246